In [1]:
data_path = '../dataset'

In [2]:
import os

In [3]:
for path in os.listdir(data_path):
    if os.path.isdir(os.path.join(data_path, path)):
        print(os.path.join(data_path, path))

../dataset/test
../dataset/train


In [4]:
train_data_path = os.path.join(data_path, 'train')
test_data_path = os.path.join(data_path, 'test')

In [5]:
train_classes = dict()

for path in sorted(os.listdir(train_data_path)):
    if os.path.isdir(os.path.join(train_data_path, path)):
        train_classes.setdefault(len(train_classes), path)
        
train_classes

{0: 'chicken_curry',
 1: 'chicken_wings',
 2: 'fried_rice',
 3: 'grilled_salmon',
 4: 'hamburger',
 5: 'ice_cream',
 6: 'pizza',
 7: 'ramen',
 8: 'steak',
 9: 'sushi'}

In [6]:
test_classes = dict()

for path in sorted(os.listdir(test_data_path)):
    if os.path.isdir(os.path.join(test_data_path, path)):
        test_classes.setdefault(len(test_classes), path)
        
test_classes

{0: 'chicken_curry',
 1: 'chicken_wings',
 2: 'fried_rice',
 3: 'grilled_salmon',
 4: 'hamburger',
 5: 'ice_cream',
 6: 'pizza',
 7: 'ramen',
 8: 'steak',
 9: 'sushi'}

In [7]:
import torch

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device

device(type='cuda')

In [8]:
torch.cuda.get_device_name(0)

'GeForce GTX 1070'

In [9]:
torch.cuda.empty_cache()

In [10]:
from torchvision import transforms as T

In [11]:
train_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [12]:
val_transform = T.Compose([
    T.Resize((256,256)),
    T.CenterCrop((224,224)),
    T.ToTensor(),
    T.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])

In [13]:
from torchvision.datasets import ImageFolder

In [14]:
train_dataset = ImageFolder(
    root=train_data_path,
    transform=train_transform
)

In [15]:
train_dataset

Dataset ImageFolder
    Number of datapoints: 750
    Root location: ../dataset/train
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               RandomVerticalFlip(p=0.5)
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [16]:
test_dataset = ImageFolder(
    root=test_data_path,
    transform=val_transform
)

In [17]:
test_dataset

Dataset ImageFolder
    Number of datapoints: 2500
    Root location: ../dataset/test
    StandardTransform
Transform: Compose(
               Resize(size=(256, 256), interpolation=PIL.Image.BILINEAR)
               CenterCrop(size=(224, 224))
               ToTensor()
               Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
           )

In [18]:
from torch.utils.data import DataLoader

In [19]:
BATCH_SIZE = 16

In [20]:
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=True
)

In [22]:
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    num_workers=0,
    shuffle=False
)

In [23]:
from torchvision import models

In [24]:
model = models.resnet18(pretrained=True)

In [25]:
for param in model.parameters():
    param.requires_grad = False

In [26]:
import torch.nn as nn

In [27]:
n_inputs = model.fc.in_features

In [28]:
n_classes = 10

In [29]:
# sequential_layers = nn.Sequential(
#     nn.Linear(n_inputs, 256),
#     nn.ReLU(),
#     nn.Dropout(.2),
#     nn.Linear(256, 128),
#     nn.ReLU(),
#     nn.Dropout(.2),
#     nn.Linear(128, n_classes),
#     nn.LogSoftmax(dim=1)
# )

In [30]:
sequential_layers = nn.Linear(n_inputs, n_classes)

In [31]:
model.fc = sequential_layers

In [32]:
model = model.to('cuda')

In [33]:
import torch.optim as optim

In [34]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(model.parameters(), lr=1e-3, momentum=.9)

scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=7, gamma=.1)

In [35]:
loaders = {
    'train': train_loader,
    'val': test_loader
}

In [36]:
from copy import deepcopy

In [37]:
dataset_sizes = {
    'train': len(train_dataset),
    'val': len(test_dataset)
}

In [38]:
%%time

EPOCHS = 20

for epoch in range(1, EPOCHS+1):
    running_loss = .0
    running_corrects = .0
    best_acc = .0
    print(f"\nEpoch {epoch}/{EPOCHS}\n{'='*25}")
    for phase in ['train', 'val']:
        if phase == 'train': model.train()
        if phase == 'val': model.eval()
        for inputs, labels in loaders[phase]:
            inputs, labels = inputs.to(device), labels.to(device)

            optimizer.zero_grad()

            with torch.set_grad_enabled(phase == 'train'):
                outputs = model(inputs)
                _, preds = torch.max(outputs, 1)
                loss = criterion(outputs, labels)
                
                if phase == 'train':
                    loss.backward()
                    optimizer.step()

            running_loss += loss.item() * inputs.size(0)
            running_corrects += torch.sum(preds == labels)
        epoch_loss = running_loss / dataset_sizes[phase]
        epoch_acc = running_corrects.double() / dataset_sizes[phase]
        if phase == 'train': scheduler.step()
        if phase == 'val' and epoch_acc > best_acc:
            best_corrects = epoch_acc
            best_model_weights = deepcopy(model.state_dict())
        print(f"Loss ({phase}): {epoch_loss}, Acc ({phase}): {epoch_acc}")


Epoch 1/20
Loss (train): 2.258432824452718, Acc (train): 0.20533333333333334
Loss (val): 2.584778984832764, Acc (val): 0.446

Epoch 2/20
Loss (train): 1.7865686108271281, Acc (train): 0.4333333333333333
Loss (val): 2.0458528471946718, Acc (val): 0.6996

Epoch 3/20
Loss (train): 1.4709057315190632, Acc (train): 0.5946666666666667
Loss (val): 1.7325504223823547, Acc (val): 0.8276

Epoch 4/20
Loss (train): 1.2983958253860473, Acc (train): 0.6346666666666666
Loss (val): 1.538426159954071, Acc (val): 0.8632000000000001

Epoch 5/20
Loss (train): 1.2044528261820475, Acc (train): 0.6493333333333333
Loss (val): 1.4152774377822877, Acc (val): 0.9

Epoch 6/20
Loss (train): 1.0943286021550496, Acc (train): 0.7026666666666667
Loss (val): 1.3519504529953004, Acc (val): 0.894

Epoch 7/20
Loss (train): 0.9874441442489624, Acc (train): 0.7213333333333333
Loss (val): 1.2409951554298402, Acc (val): 0.9388000000000001

Epoch 8/20
Loss (train): 0.9673540533383688, Acc (train): 0.7346666666666667
Loss (val

In [39]:
torch.save(best_model_weights, 'foodnet.pth')